Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Tutorial: Load demo data and enrich it with NOAA ISD Weather data.

In this tutorial, you load the demo data (a parquet file in Azure Blob), check the data schema, enrich it with NOAA ISD Weather data.

Prerequisites:
> You must install the PyPi package on the cluster:
> * pandas version must be 0.23.0 or above
> * azureml-contrib-opendatasets

Learn how to:
> * Load the demo data from Azure Blob
> * Check the demo data schema
> * Initialize NoaaIsdWeather class to load weather data
> * Enrich the demo data with weather data
> * Display the joined result annd stats

## Install azureml-opendatasets package

In [4]:
!pip uninstall -y azureml-opendatasets
!pip install azureml-opendatasets

## Define a DemoData class to load demo parquet from Azure Blob

In [6]:
from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq
from io import BytesIO

class DemoData:
    def __init__(self):
        self.blob_account_name = "azureopendatastorage"
        self.blob_container_name = "tutorials"
        self.blob_relative_path = 'noaa_isd_weather/demo.parquet'

    def to_pandas_dataframe(self):
        blob_service = BlockBlobService(account_name=self.blob_account_name)
        byte_stream = BytesIO()
        blob = blob_service.get_blob_to_stream(
            container_name=self.blob_container_name,
            blob_name=self.blob_relative_path,
            stream=byte_stream)

        return pq.read_table(source=byte_stream).to_pandas()

## Initialize a DemoData instance and load the pandas DataFrame and check the schema

In [8]:
df = DemoData().to_pandas_dataframe()
df.dtypes

Out[ 2 ]: 
datetime datetime64[ns]
lat float64
long float64
stations.city object
count int32
stations.dock_count int32
dtype: object

## Display the top 5 rows in the demo data dataframe

In [10]:
df.head(5)

Out[ 3 ]: 
 datetime lat ... count stations.dock_count
0 2015-05-01 37.787152 ... 28 15
1 2015-05-02 37.787152 ... 5 15
2 2015-05-03 37.787152 ... 11 15
3 2015-05-04 37.787152 ... 24 15
4 2015-05-05 37.787152 ... 24 15

[5 rows x 6 columns]

## Initialize NoaaIsdWeather class, get the enricher from it and enrich demo data
For weather data, due to size, by default we allow reading from the last month if multiple months are passed.If you want to load more, please refer to `04-nyc-taxi-join-weather-in-pandas.ipynb.ipynb` under this folder for how.

The logic for join:

The join logic for Pandas version is using cKDTree to accelerate the speed of the process. We gather the public weather dataset as long/lat point array, pass it to create cKDTree. Then gather the customer dataset as long/lat point array, pass it to cKDTree query function, to find the closest point in cKDTree. After querying cKDTree, we join public weather dataset with customer dataset by the querying result, then grant ranking group id.

In [12]:
# This is a package in preview.

from azureml.opendatasets.accessories.location_data import LatLongColumn
from azureml.opendatasets.accessories.location_time_customer_data import LocationTimeCustomerData
from azureml.opendatasets import NoaaIsdWeather
from datetime import datetime


_customer_data = LocationTimeCustomerData(df, LatLongColumn('lat', 'long'), 'datetime')
weather = NoaaIsdWeather(
    cols=["temperature", "windSpeed", "seaLvlPressure"],
    start_date=datetime(2015, 5, 1, 0, 0),
    end_date=datetime(2015, 5, 31, 23, 59))
weather_enricher = weather.get_enricher()
joined_data = weather_enricher.enrich_customer_data_with_agg(
    customer_data_object=_customer_data,
    location_match_granularity=5,
    time_round_granularity='day',
    agg='avg')

ActivityStarted, get_enricher
ActivityCompleted: Activity=get_enricher, HowEnded=Success, Duration=1.65 [ms]
ActivityStarted, enrich_customer_data_with_agg
ActivityStarted, enrich
Target paths: ['/year=2015/month=5/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading ISDWeather/year=2015/month=5/part-00001-tid-2198075741767757560-e3eb994e-d560-4dfc-941e-0aae74c8d9ed-93.c000.snappy.parquet under container isdweatherdatacontainer
Done.
ActivityStarted, _get_closest_location_kdTree
ActivityCompleted: Activity=_get_closest_location_kdTree, HowEnded=Success, Duration=26.44 [ms]
ActivityCompleted: Activity=enrich, HowEnded=Success, Duration=36200.08 [ms]
ActivityCompleted: Activity=enrich_customer_data_with_agg, HowEnded=Success, Duration=36201.54 [ms]

## Display the top 10 rows of the joined result

In [14]:
joined_data.data.head(10)

Out[ 6 ]: 
 datetime lat ... windSpeed seaLvlPressure
0 2015-05-01 37.787152 ... 3.123711 1010.11125
1 2015-05-02 37.787152 ... 3.690777 1011.76125
2 2015-05-03 37.787152 ... 3.579512 1011.81125
3 2015-05-04 37.787152 ... 4.349289 1014.19500
4 2015-05-05 37.787152 ... 4.650739 1014.70625
5 2015-05-06 37.787152 ... 5.280488 1012.26000
6 2015-05-07 37.787152 ... 4.492424 1008.61000
7 2015-05-08 37.787152 ... 2.853769 1009.97250
8 2015-05-09 37.787152 ... 3.574510 1017.74359
9 2015-05-10 37.787152 ... 4.777209 1018.53500

[10 rows x 9 columns]

## Check the stats of joined result

In [16]:
joined_data.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1850 entries, 0 to 1849
Data columns (total 9 columns):
datetime 1850 non-null datetime64[ns]
lat 1850 non-null float64
long 1850 non-null float64
stations.city 1850 non-null object
count 1850 non-null int32
stations.dock_count 1850 non-null int32
temperature 1850 non-null float64
windSpeed 1850 non-null float64
seaLvlPressure 1850 non-null float64
dtypes: datetime64[ns](1), float64(5), int32(2), object(1)
memory usage: 130.1+ KB